$$ 
\newcommand{\prover}{\mathcal{P}}
\newcommand{\verifier}{\mathcal{V}}
\newcommand{\Mat}[1]{\mathbf{{#1}}}
\newcommand{\Fx}[1]{\mathbb{F}^{(\preceq {#1})}}
\newcommand{\mle}[1]{\widetilde{#1}}
\newcommand{\eq}{\textsf{eq}}
\newcommand{\concat}{+\kern-1.3ex+\kern0.8ex}
$$

# Spartan: Efficient and general-purpose zkSNARKs without trusted setup

Authors: Srinath Setty <br/>
ePrint: [2019/550](https://eprint.iacr.org/2019/550.pdf)

## Introduction

Spartan is a transparent [SNARK](https://en.wikipedia.org/wiki/Non-interactive_zero-knowledge_proof) for R1CS [constraint satisfaction problem](https://en.wikipedia.org/wiki/Constraint_satisfaction_problem). R1CS is a popular NP-Complete problem that enjoys extensive tooling support from the compiler community (or so I've been told).

An R1CS instance consists of
1. Three $n \times m$ _sparse_ matrices $\Mat{A}, \Mat{B}, \Mat{C} \in \mathbb{F}^{n\times m}$ that encode the circuit/constraints representing the CSP, and 
2. An arbitrary $m$ dimensional vector $\vec{w} \in \mathbb{F}^m$.

The vector $\vec{w}$ is _considered a witness_ to the NP statement:

$$
    (\Mat{A}\cdot \vec{w}) \circ(\Mat{B}\cdot \vec{w}) \stackrel{?}{=} \Mat{C}\cdot \vec{w}
$$

if the above equation is satisfied. <u>Notation</u>: $\circ$ is the Hadamard (element wise) product. (R1CS captures arithmetic circuits with fan-in $2$ multiplication gates, and arbitrary fan-in addition gates.)

Without loss of generality, we assume $m = n$ in the rest of the document and treat $\Mat{A},\Mat{B},\Mat{C} \in \mathbb{F}^{m\times m}$, and $\vec{w} \in \mathbb{F}^m$ (if $m\neq n$, then $\Mat{A},\Mat{B},\Mat{C}$ and $\vec{w}$ can be padded appropriately to get the desired result). Furthermore, a matrix $\Mat{M} \in \mathbb{F}^{m\times m}$ is defined to be _sparse_ if the number of non-zero entries, call it $\ell$, is sublinear in $m^2$ (for example, say $\ell \in O(m)$).

Spartan -- as described here -- is a public-coin Polynomial _Interactive_ Oracle Proof (PIOP) that can be converted into a SNARK using using Fiat-Shamir heuristics.  

The remarkable features of Spartan are:
* the prover's time is _strictly linear_ in $\ell$, and 
* the verifier's time is polylogarithmic in $\ell$, i.e., $O(\log(\ell)^c)$ for some fixed constant $c$. 

Note that in a typical use case (e.g. google's
[zk-longfellow](https://github.com/google/longfellow-zk.git) for legacy identity
verification), the value of $\ell$ is of the order of $2^{20}$. Unless the
prover time is linear (or at worst quasi-linear) in $\ell$, it won't be
computationally possible to build practical system on existing hardware (e.g., iPhone or Mac OSX).

### Sumcheck: A protocol for $\forall \to \exists$ quantifier reduction

The goal of _any_ R1CS prover is to prove that _for all_ rows: $(\Mat{A}\cdot \vec{w}){\color{red} \circ}(\Mat{B}\cdot \vec{w}) - (\Mat{C}\cdot \vec{w}) = 0$ holds. However, since the verifier's time is required to be poly-logarithmic in the witness length $m = |\vec{w}|$, it cannot check the validity of each row individually! Therefore, the fist design choice in building any R1CS SNARK is to decide how to transform a _logical claim_ about "**for all rows**" to a claim about "**for only a few** (polylog or constant) number of rows," without sacrificing too much in soundness.

The simplest strategy one could think of is that the verifier _randomly_ checks a polylog number of rows, say $\log(m)^{\kappa}$ rows for some integer $\kappa$. This strategy, however, allows a malicious prover to cheat with probability $1 - \frac{\log(m)^\kappa}{m} \approx 1 - o(\frac{1}{m})$ (for large $m$). For large values of $m$ (e.g., $m \approx 2^{20}$), this allows the prover to cheat with almost certainty! Ideally, one would like the cheating probability to  grow -- at best -- poly-logarithmically in the size of the problem instance (ideally size of the witness), and be adjustable according to some security parameters $\lambda$.

Sumcheck [[LFKN'92](https://lance.fortnow.com/papers/files/ip.pdf)] is an elegant protocol for _reducing_ a claim about the _sum_ of **all** evaluations of a multivariate polynomial over a per-defined set (typically the boolean hypercube), to a claim about **one** random evaluation of the same polynomial by the verifier. In particular, if $f(\vec{x}) \in \mathbb{F}^{(\preceq d)}[x_1,\cdots, x_s]$ is a $s$-variate polynomial with maximal individual degree $d$, then Sumcheck is a reduction from a claim of the form 

$$
F \stackrel{?}{=} \sum_{\vec{b} \in \{0,1\}^s} f(\vec{b})
$$

to a claim of the form 

$$
G \stackrel{?}{=} f(r_1,\cdots, r_s)\quad \text{where}\; r_i \xleftarrow{\$} \mathbb{F}
$$

The soundness loss (i.e., prover's cheating probability) in the above reduction is $1 - \left(1 - \frac{d}{|\mathbb{F}|} \right)^s \approx s \cdot \frac{d}{|\mathbb{F}|}$. 


Spartan's meta strategy is to reduce an R1CS claim to a Sumcheck claim with $s = O(\log |m|)$ and $d \leq 3$. Unlike the random row selection strategy discussed before, Sumcheck provides much better soundness guarantees.

**Notation (Sumcheck)**:
>
> Let $\vec{r}_\Sigma := (r_1,\cdots, r_s) \in \mathbb{F}^s$ be the verifier's random challenges during the execution of Sumcheck protocol. Let $G$ be the prover's claim about the value of $f(\vec{r}_\Sigma)$ at the end of Sumcheck protocol. This document uses the following notation to capture this information symbolically: $$F \stackrel{?}{=} \sum_{\vec{b} \in \{0,1\}^s} f(\vec{b}) \stackrel{\vec{r}_\Sigma}{\implies} G \stackrel{?}{=} f(\vec{r}_\Sigma)$$


### Spartan Design Considerations

There are two challenges in (Karp) reducing an R1CS instance into a Sumcheck instance: One for the prover and one for the verifier-
1. **Prover**: How to arithmetize an R1CS instance such that the total cost of proof generation is _concretely_ at-worst quasi-linear in the witness size $m$ (not $m^2$, which is the size of matrices $\Mat{A}, \Mat{B},$ and $\Mat{C}$ ). 
2. **Verifier**: How to validate the claim  $G \stackrel{?}{=} f(r_1,\cdots, r_s)$ in polylog $m$ time. Note that if $s = \lceil\log m \rceil$, then $f$ has $O(2^s) = O(m)$ coefficients and directly evaluating $f$ requires at least reading all the coefficients of $f$, which can't be made sublinear in $m$.

In Spartan, the challenge faced by the Prover is addressed by exploiting the sparse nature of R1CS matrices. In particular, if matrices $\Mat{A}, \Mat{B},$ and $\Mat{C}$ have $O(m)$ non-zero coefficients, then Spartan specifies an arithmetization procedure with time complexity $O(m)$ instead of $O(m^2)$. For the verifier, Spartan relies on a Polynomial Commitment Scheme (PCS) such as [WHIR](https://eprint.iacr.org/2024/1586) to bring down the verifier's cost to sublinear complexity.

#### Matrix Vector Arithmetization

Let $s = \lceil \log m \rceil$ and define $[m] := \{ 0, \cdots, m-1 \}$. Given an index $k \in [m]$ let $\vec{k} \in \{0,1\}^s$ denote the binary decomposition of $k$ according to some fixed convention (e.g., MSB first). **N. B.**:  $\vec{k}$ is an ordered tuple, while $k$ (without arrow on head) is a non-negative integer less than $m$. (For example, if $m = 13$, then $s := \lceil \log 13 \rceil = 4$ and $\vec{3} := (0, 0, 1, 1 )$ according to MSB first decomposition.)

Given a matrix $ \Mat{M} \in \mathbb{F}^{m\times m} $, it's $(i,j)$-th entry $\Mat{M}[i,j]$ can be interpreted as the _value_ of the unique multilinear polynomial $\mle{M}(\vec{x}, \vec{y}) \in \Fx{1}[x_1,\cdots, x_s;\;y_1,\cdots, y_s]$, i.e., 

$$
\mle{M}(\vec{i},\vec{j}) = \Mat{M}[i,j]
$$

This unique multilinear polynomial (henceforth MLE), can be expressed as 

$$
\mle{M}(\vec{x},\vec{y}) := \sum_{i \in [m]}\sum_{j \in [m]} \Mat{M}[i,j]\cdot\eq(\vec{x}, \vec{i})\cdot \eq(\vec{y}, \vec{j})
$$

where $\eq(\vec{x}, \vec{a}) \in \Fx{1}[x_1,\cdots,x_s]$ is the Lagrange basis
$$
\eq(\vec{x}, \vec{a}) := \prod_{i \in \{1,\cdots, s\}} \left[x_i\cdot a_i + (1-x_i)\cdot(1-a_i) \right] = \begin{cases} 1 & \text{if}\; (\vec{x} = \vec{a})\wedge (\vec{x}, \vec{a} \in \{0,1\}^s)  \\ 0 & \text{if}\; (\vec{x} \neq \vec{a})\wedge (\vec{x}, \vec{a} \in \{0,1\}^s) \\ \in \mathbb{F}& \text{otherwise} \end{cases}
$$

Similarly, given the witness vector $\vec{w} \in \mathbb{F}^m$, its MLE can be expressed as 

$$
\mle{w}(\vec{y}) = \sum_{ i \in [m]} \vec{w}[i]\cdot \eq(\vec{y}, \vec{i}) \in \Fx{1}[y_1,\cdots,y_s]
$$

Given this arithmetization, notice that $\forall (i,j) \in [m]\times[m]$:

$$
 \Mat{M}[i,j]\cdot \vec{w}[j] = \mle{M}(\vec{i}, \vec{j})\cdot \mle{w}(\vec{j})
$$

#### R1CS Arithmetization

**Theorem**
>
> Given the Matrix-Vector arithmetization over the boolean hypercube $\mathcal{B}_s := \{0,1\}^s$ where $s = \lceil \log |\vec{w} | \rceil$, an R1CS instance $ (\Mat{A},\Mat{B},\Mat{C}, \vec{w}) $ with constraint equation
> $$ (\Mat{A}\cdot \vec{w}) \circ (\Mat{B}\cdot \vec{w}) \stackrel{?}{=} \Mat{C}\cdot \vec{w}$$
>
>is satisfiable if and only if the following polynomial $\hat{f}(\vec{x}) \in \Fx{2}[x_1,\cdots,x_s]$ is identically zero for all values of $\vec{x} \in \mathcal{B}_s$:
> $$\hat{f}(\vec{x}) := \left( \sum_{y \in [m]} \mle{A}(\vec{x}, \vec{y})\cdot \mle{w}(\vec{y}) \right )\cdot \left( \sum_{y \in [m]} \mle{B}(\vec{x}, \vec{y})\cdot \mle{w}(\vec{y}) \right ) - \left( \sum_{y \in [m]} \mle{C}(\vec{x}, \vec{y})\cdot \mle{w}(\vec{y}) \right ) $$
>
> **NOTE**: $\hat{f}(\vec{x})$ as a polynomial function is only zero on $\mathcal{B}_s$, not the entire domain $\mathbb{F}^s$. Also note that $\vec{w} =0$ satisfies all R1CS instances.

**Proof**

> $(\Rightarrow)$:
> If the R1CS instance is satisfiable, then $$
\forall i \in [m]: \quad \left(\sum_{j \in [m]}\Mat{A}[i,j]\cdot \vec{w}[j]\right)\cdot\left(\sum_{j \in [m]}\Mat{B}[i,j]\cdot \vec{w}[j]\right) - \left(\sum_{j \in [m]}\Mat{C}[i,j]\cdot \vec{w}[j] \right) = 0 $$
> By construction, $\forall (i,j) \in [m]\times[m]: \mle{M}(\vec{i}, \vec{j})\cdot \mle{w}(\vec{j}) = \Mat{M}[i,j]\cdot \vec{w}[j]$, therefore $$ \begin{aligned}
\forall i \in [m]:\quad \hat{f}(\vec{i}) &= \left(\sum_{j \in [m]} \mle{A}(\vec{i}, \vec{j})\cdot \mle{w}(\vec{j}) \right )\cdot \left( \sum_{j \in [m]} \mle{B}(\vec{i}, \vec{j})\cdot \mle{w}(\vec{j}) \right ) - \left( \sum_{j \in [m]} \mle{C}(\vec{i}, \vec{j})\cdot \mle{w}(\vec{i}) \right ) \\ 
&= \left(\sum_{j \in [m]}\Mat{A}[i,j]\cdot \vec{w}[j]\right)\cdot\left(\sum_{j \in [m]}\Mat{B}[i,j]\cdot \vec{w}[j]\right) - \left(\sum_{j \in [m]}\Mat{C}[i,j]\cdot \vec{w}[j] \right) = 0\end{aligned}$$ 

>
> $(\Leftarrow)$: Given four _multilinear_ polynomials $(\mle{A}, \mle{B}, \mle{C}, \mle{w})$, that satisfies
> $f(\vec{i}) = 0$ for all $i \in [m]$, our goal is to prove that this corresponds to a satisfiable R1CS instance $(\Mat{A}, \Mat{B}, \Mat{C}, \vec{w})$. Since $\forall i \in [m]: f(\vec{i}) = 0$ implies there exist $m$ equations of the form: $$0 = \left(\sum_{j \in [m]}\mle{A}(\vec{i},\vec{j})\cdot\mle{w}(\vec{j})\right)\cdot \left(\sum_{j \in [m]}\mle{B}(\vec{i},\vec{j})\cdot\mle{w}(\vec{j})\right) - \left(\sum_{j \in [m]}\mle{C}(\vec{i},\vec{j})\cdot\mle{w}(\vec{j})\right)$$
>
> These $m$ equations can be interpreted as a satisfiable Matrix-Vector product equation for R1CS.
>

For notational convenience, the following intermediate polynomials are defined

**Notation**
> 
> 
> $$ \begin{aligned}  \overline{A}(\vec{x}) &:= \sum_{\vec{y} \in \{0,1\}^s}\mle{A}(\vec{x}, \vec{y})\cdot \mle{w}(\vec{y}) \quad &\in& \quad \Fx{1}[x_1,\cdots,x_s]\\ \overline{B}(\vec{x}) &:= \sum_{\vec{y} \in \{0,1\}^s}\mle{B}(\vec{x}, \vec{y})\cdot \mle{w}(\vec{y}) \quad &\in& \quad \Fx{1}[x_1,\cdots,x_s] \\ \overline{C}(\vec{x}) &:= \sum_{\vec{y} \in \{0,1\}^s}\mle{C}(\vec{x}, \vec{y})\cdot \mle{w}(\vec{y}) \quad &\in& \quad \Fx{1}[x_1,\cdots,x_s] \\ \hat{f}(\vec{x}) &\;= \overline{A}(\vec{x})\cdot\overline{B}(\vec{x}) - \overline{C}(\vec{x}) \quad &\in& \quad  \Fx{2}[x_1,\cdots, x_s] \end{aligned}$$
>


**Remark**: Since $(\overline{A}, \overline{B}, \overline{C})$ are multilinear, the individual degree of each $x_i$ in $\hat{f}(x_1,\cdots, x_s)$ can be at most $2$. Furthermore, unless $\overline{A}$ or $\overline{B}$ is a constant polynomial, $\hat{f}(\vec{x})$ will not be an identically zero polynomial since $\overline{A}(\vec{x})\cdot\overline{B}(\vec{x})$ is multi-quadratic while $\overline{C}(\vec{x})$ is multilinear. 

In summary, with overwhelming probability, $\hat{f}$ is not an identically zero polynomial for a randomly selected R1CS instance.

#### Naïve Spartan

Given such an arithmetized polynomial $\hat{f}$, it's tempting to think that the verifier can directly invoke [Schwartz-Zippel](https://en.wikipedia.org/wiki/Schwartz%E2%80%93Zippel_lemma) lemma by randomly sampling $\vec{r} \xleftarrow{\$} \mathbb{F}^s$ and validating that

$$ \hat{f}(\vec{r}) := \overline{A}(\vec{r})\cdot \overline{B}(\vec{r}) - \overline{C}(\vec{r}) \stackrel{?}{=} 0
$$

To do that, since $\overline{A}, \overline{B},$ and $\overline{C}$ are defined as a sum over the boolean hypercube, the Prover $\mathcal{P}$ and the Verifier $\mathcal{V}$ could check the validity of the claim above as follows:

##### Failed Attempt

1. $\mathcal{V} \longrightarrow \mathcal{P}$: Verifier samples a random challenge $\vec{r} \xleftarrow{\$} \mathbb{F}^s$ and sends $\vec{r}$ to the Prover.
2. $\mathcal{P} \longrightarrow \mathcal{V}$: Prover evaluates $$\begin{aligned}v_A &:= \overline{A}(\vec{r})\\ v_B &:= \overline{B}(\vec{r})\\ v_C &:= \overline{C}(\vec{r})\end{aligned}$$
   and sends $(v_a, v_b, v_c) \in \mathbb{F}^3$ to the verifier.
3. $\mathcal{V} \longleftrightarrow \mathcal{P}$: Verifier checks the following, and returns $\textsf{reject}$ if any of the checks fails:
   * $v_A\cdot v_B \stackrel{?}{=} v_C$.
   * Even if the claim above validates, the Verifier doesn't know if the Prover's claim $(v_A, v_B, v_C) \stackrel{?}{=} (\overline{A}(\vec{r}), \overline{B}(\vec{r}), \overline{C}(\vec{r}))$ is valid. To validate these secondary claims, $\mathcal{P}$ and $\mathcal{V}$ can use Sumcheck as follows:
   * $\mathcal{P}$ and $\mathcal{V}$ could run three independent copies of Sumcheck protocol with verifier randomness $(\vec{r}_\Sigma^A, \vec{r}_\Sigma^B, \vec{r}_\Sigma^C)$ to reduce the claim from $(\overline{A}(\vec{r}), \overline{B}(\vec{r}), \overline{C}(\vec{r}))$, to a claim about the polynomial evaluation with values $(\hat{v}_A, \hat{v}_B, \hat{v}_C)$, i.e., $$ \begin{aligned} v_A &\stackrel{?}{=} \sum_{\vec{b} \in \mathcal{B}_s}\mle{A}(\vec{r}, \vec{b})\cdot \mle{w}(\vec{b}) \stackrel{\vec{r}_\Sigma^A}{\implies}  \hat{v}_A \stackrel{?}{=} \mle{A}(\vec{r}, \vec{r}_\Sigma^A)\cdot \mle{w}(\vec{r}_\Sigma^A)  \\ v_B &\stackrel{?}{=} \sum_{\vec{b} \in \mathcal{B}_s}\mle{B}(\vec{r}, \vec{b})\cdot \mle{w}(\vec{b}) \stackrel{\vec{r}_\Sigma^B}{\implies}  \hat{v}_B \stackrel{?}{=} \mle{B}(\vec{r}, \vec{r}_\Sigma^B)\cdot \mle{w}(\vec{r}_\Sigma^B)  \\ v_C &\stackrel{?}{=} \sum_{\vec{b} \in \mathcal{B}_s}\mle{C}(\vec{r}, \vec{b})\cdot \mle{w}(\vec{b}) \stackrel{\vec{r}_\Sigma^C}{\implies}  \hat{v}_C \stackrel{?}{=} \mle{C}(\vec{r}, \vec{r}_\Sigma^C)\cdot \mle{w}(\vec{r}_\Sigma^C) \end{aligned}$$ where the symbol $\stackrel{\vec{r}_\Sigma^{Z}}{\implies}$ means *Sumcheck reduction* using verifier randomness $\vec{r}_\Sigma^{Z}$. 
   * Assuming Verifier has oracle access to $(\mle{A}, \mle{B}, \mle{C}, \mle{w})$, it will need to make $6$ oracle queries $\left \{\mle{A}(\vec{r}, \vec{r}_\Sigma^A), \mle{B}(\vec{r}, \vec{r}_\Sigma^B), \mle{C}(\vec{r}, \vec{r}_\Sigma^C), \mle{w}(\vec{r}_\Sigma^A),  \mle{w}(\vec{r}_\Sigma^B), \mle{w}(\vec{r}_\Sigma^C) \right\}$ to validate claims about $(\hat{v}_A, \hat{v}_B, \hat{v}_C)$.

Alas, **this scheme does not work** because of a technicality! The claim that $\hat{f}(\vec{x}) = 0$, is only valid over the boolean hypercube $\{0,1\}^s$, not for any randomly sampled point $\vec{r}$ from the entire domain $\mathbb{F}^s$.

Spartan solves this problem by multi-linearizing $\hat{f}(\vec{x})$. Given, $\hat{f} \in \Fx{2}[x_1,\cdots,x_s]$ consider another polynomial $\mathcal{Q}(\vec{\tau})$ with the following properties:
1. $\mathcal{Q}(\vec{\tau})$ is multilinear, i.e., $\mathcal{Q}(\vec{\tau}) \in \Fx{1}[\tau_1,\cdots, \tau_s]$, and
2. The value of $\hat{f}$ and $\mathcal{Q}$ match exactly on the boolean hypercube, i.e., $\forall \vec{b} \in \{0,1\}^s:\quad \mathcal{Q}(\vec{b}) = \hat{f}(\vec{b})$

Since $\mathcal{Q}$ is multilinear, it's easy to efficiently compute $\mathcal{Q}$ from $\hat{f}$ using Lagrange interpolation as follows:

$$
\mathcal{Q}(\vec{\tau}) := \sum_{\vec{x} \in \{0,1\}^s} \hat{f}(\vec{x})\cdot \eq(\vec{b}, \vec{\tau}) \quad \in \quad \Fx{1}[\tau_1,\cdots, \tau_s]
$$

Based on the previous theorem, an R1CS instance is satisfiable _if and only if_ $\hat{f}(\vec{x})$ is **identically zero** over the boolean hypercube $\{0,1\}^s$. Extending this result to $\mathcal{Q}(\vec{\tau})$, its easy to see that $\mathcal{Q}(\vec{\tau})$ is an arithmetization of a satisfying R1CS instance, _if and only if_ $\mathcal{Q}(\vec{\tau})$ is _the_ **identically zero** polynomial $\mathcal{Q}(\vec{\tau}) = 0 \in \Fx{1}[\tau_1,\cdots,\tau_s]$. (Recall, a multilinear polynomial is uniquely determined by the values it takes on its basis polynomials.)

**Notation** 

> $$\begin{aligned} \mathcal{G}(\vec{x}, \vec{\tau}) &:= \hat{f}(\vec{x})\cdot \eq(\vec{x}, \vec{\tau}) &\in& \quad\Fx{3}[x_1,\cdots,x_s;\;\tau_1,\cdots, \tau_s] \\ 
\mathcal{Q}(\vec{\tau}) &:= \sum_{\vec{x} \in \{0,1\}^s} \mathcal{G}(\vec{x}, \vec{\tau}) &\in& \quad \Fx{1}[\tau_1,\cdots, \tau_s] \end{aligned}$$

The figure below shows a functionally correct instantiation of Spartan, assuming the Prover and the Verifier have access to an _arbitrary-degree_ multivariate polynomial commitment scheme (PCS). The goal of the Prover $\mathcal{P}$ is to convince the Verifier $\mathcal{V}$ that $\mathcal{Q}(\vec{\tau})$ is the identically zero polynomial. (Note that the maximum individual degree of $\hat{f}$ is $2$, so a multilinear polynomial commitment scheme such as WHIR will not work in this naïve instantiation.)

The details of the protocol are summarized below (See <a href="#spartan-naive-image">figure below</a> for a pictorial description):

> **Goal**: Prover $\mathcal{P}$ to the convince the Verifier $\mathcal{V}$ that $\mathcal{Q}(\vec{\tau})$ is zero on the entire domain $\mathbb{F}^s$. $\mathcal{P}$ and $\mathcal{V}$ proceed as follows:
> 
>  $\mathcal{P} \longrightarrow \mathcal{V}$: Using a suitable Polynomial Commitment Scheme (PCS), the Prover sends a commitment of $\hat{f}$, say $\textsf{commit}(\hat{f})$, to the Verifier.
> 
> $\mathcal{V} \longrightarrow \mathcal{P}$: Verifier uniformly samples $s$ random field elements $\vec{r}_\tau \xleftarrow{\$} \mathbb{F}^s$ and sends it to the Prover. Verifier expects the Prover to use Sumcheck to prove the claim: $$0 \stackrel{?}{=} \sum_{\vec{b}\in \{0,1\}^s} \hat{f}(\vec{b})\cdot \eq(\vec{b}, \vec{r}_\tau)$$ 
>
> $\mathcal{V} \longleftrightarrow \mathcal{P}$: Prover and Verifier exchange Sumcheck messages, with Verifier sending $\vec{r}_\Sigma$ random challenges, and Prover claiming at the last round: $$G \stackrel{?}{=} \hat{f}(\vec{r}_\Sigma)\cdot \eq(\vec{r}_\Sigma, \vec{r}_\tau)$$


> In order to validate the above claim, the Verifier needs to compute $\hat{f}(\vec{r}_\Sigma) \cdot \eq(\vec{r}_\Sigma, \vec{r}_\tau)$ "on its own." Since $\eq(\vec{x}, \vec{\tau})$ is public, verifier can locally (and efficiently) compute $\eq(\vec{r}_\Sigma, \vec{r}_\tau)$, however, it cannot compute $\hat{f}(\vec{r}_\Sigma)$ directly since it only has oracle access to $\hat{f}$, via $\textsf{commit}(\hat{f})$. To compute $\hat{f}(\vec{r}_\Sigma)$, the Verifier will use the polynomial commitment scheme as follows:


> $\mathcal{V} \longrightarrow \mathcal{P}$: Verifier instructs Prover to evaluate $\hat{f}(\vec{r}_\Sigma)$ and provide a proof of correct evaluation. 
>
> $\mathcal{P} \longleftrightarrow \mathcal{V}$: Prover evaluates $ F = \hat{f}(\vec{r}_\Sigma)$ and follows PCS opening proof protocol, to generate a proof $\pi$ of correct evaluation. Note that $\hat{f}$ was committed by the Prover _before_ the verifier picked its Sumcheck randomness $\vec{r}_\Sigma$. Therefore, assuming the PCS commitment is binding, the prover cannot open $\hat{f}(\vec{r}_\Sigma)$ in two different ways with more than non-negligible probability. 
>
> $\mathcal{V}$: The Verifier accepts if the quadruple $\left(\textsf{commit}(\hat{f}), \vec{r}_\Sigma, F, \pi \right)$ is valid evaluation proof of $$F = \hat{f}(\vec{r}_\Sigma)$$ and $$ G = F\cdot \eq(\vec{r}_\Sigma, \vec{r}_\tau)$$

**Remark**: Over the boolean hypercube, by definition: $\vec{x} \neq \vec{y} \implies \eq(\vec{x},\vec{y}) = 0$. Outside the boolean hypercube, this is not the case, i.e., if $\vec{x}, \vec{y} \in \mathbb{F}^s\setminus \{0,1\}^s$ then $\vec{x} \neq \vec{y} \centernot\implies \eq(\vec{x},\vec{y}) = 0$. In the equation above, since $\vec{r}_\Sigma$ and $\vec{r}_\tau$ are sampled uniformly from the entire domain $\mathbb{F}^s$ -- not just the boolean hypercube -- $\eq(\vec{r}_\Sigma, \vec{r}_\tau)$ will not be identically zero, even thought w.h.p. $\vec{r}_\Sigma \neq \vec{r}_\tau$. This ensures that the prover cannot predict the value of $G$ to be $0$ with high probability, which would lead to complete soundness loss.

The figure below summarizes the protocol pictorially:

<img id="spartan-naive-image" src="./spartan/Spartan-0.drawio.svg" style="margin-left:auto; margin-right:auto"/>

#### Soundness of Naïve Spartan

The scheme described above has two sources of soundness loss (a.k.a. cheating probability): First during the Sumcheck protocol and second during PCS opening. The maximum individual degree of $\mathcal{G}(\vec{x},\vec{r}_\tau)$ is $3$, therefore, the soundness loss due to first Sumcheck is $$\frac{3}{|\mathbb{F}|}\cdot s$$

Assuming the PCS scheme has soundness loss of $\delta$, by union bound the total cheating probability of the Prover is limited to $$\frac{3}{|\mathbb{F}|}\cdot s + \delta$$

#### Time complexity

**Prover**: For the Prover, the naïve arithmetization of $\overline{A}(\vec{x},\vec{y}), \overline{B}(\vec{x},\vec{y}), \overline{C}(\vec{x},\vec{y})$ &mdash; needed for polynomial commitment &mdash; is at least quadratic (i.e., $O(m^2))$ and dominates the linear time complexity of Sumcheck.   

**Verifier**: For the Verifier, the time complexity is dominated by PCS opening proof verification. Note that even though the verifier needs to locally compute $\eq(\vec{r}_\Sigma, \vec{r}_\tau)$, because of the multiplicative nature of $\eq$, it can be computed efficiently in $O(s) = O(\log m)$ time.

### Spartan Full

While the naïve Spartan is functionally correct, it suffers from both theoretical and practical problems. Theoretically, the Prover's time is quadratic instead of linear/quasi-linear (this is addressed after the protocol description). Practically, there are three problems:

1. In real world deployments, the R1CS matrices $(\Mat{A}, \Mat{B}, \Mat{C})$ correspond to the circuit that the user _implicitly trusts_ as _the_ circuit to validate the $\mathsf{NP}$ statement. Therefore, in most practical deployments, the arithmetization of $(\Mat{A}, \Mat{B}, \Mat{C})$ is done ahead of time and the Verifier needs _trusted commitments_ to $(\mle{A}, \mle{B}, \mle{C})$ which is independent of the witness $\vec{w}$. (These trusted commitment can be trivially generated by simply signing the commitment using some post-quantum signature scheme, such as [CRYSTALS-Dilithium](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.204.pdf). Note that, as usual, trusted commitmment doesn't mean trustworthy commitment!)
2. The witness $\vec{w}$ usually consists of public input $\vec{u}$ and private input $\vec{v}$. In most use cases, the Prover would like to keep all information about $\vec{v}$ private. Furthermore, since the all zero-vector $\vec{v} = 0 \in \mathbb{F}^m$ always satisfies all R1CS instances, the Verifier should have guarantees that $\vec{w}$ has at least one non-zero entry. Spartan addresses this issue by defining $\vec{w} := \vec{u} \concat [1] \concat \vec{v}$ and artificially inserting an all ones column (i.e., $\{1,\cdots, 1\} \in \mathbb{F}^m$) in $\Mat{A}, \Mat{B}$, and $\Mat{C}$. (Here, $\concat$ operator concatenates two vector sequentially.)
3. The entire scheme should be Zero-Knowledge if needed. This requires not only Zero-Knowledge Sumcheck protocol but also [Zero-Knowledge support from PCS scheme](https://github.com/worldfnd/provekit/blob/main/sage/fri-and-friends/Zero%20Knowledge%20for%20WHIR.md).

The main idea in Spartan is to commit to "structural polynomials" $(\mle{A}, \mle{B}, \mle{C})$ and the witness polynomial $\mle{w}$ independently, and let the verifier combine them to achieve the same result as the <a href="#Na%C3%AFve-Spartan">Naïve Spartan</a>. This has the added benefit that if the prover is stateful, it can amortize the cost of commitment of $(\mle{A}, \mle{B}, \mle{C})$ over multiple runs of the protocol with different witnesses $\mle{w}$s.

The protocol runs in two stages. In the first stage, which is similar to Naïve Spartan Sumcheck claim, the Prover attempts to convince the verifier that $$\forall \vec{\tau}\in\mathbb{F}^s:\quad \mathcal{Q}(\vec{\tau}) := \sum_{\vec{x}\in \{0,1\}^s} \mathcal{G}(\vec{x},\vec{\tau}) = 0$$

However, unlike in case of Naïve Spartan, the prover cannot validate the last round of Sumcheck claims directly using a PCS. Instead, the Prover and Verifier run a second round of _batched_ Sumcheck claim followed by four PCS opening proofs to complete the validation (if the PCS supports efficient batching, the four opening proofs can be batched).

The following protocol describes the message exchange between an **honest** Prover $\mathcal{P}$ and the Verifier $\mathcal{V}$ in more detail. (If the Prover is dishonest, it's allowed to send whatever it wants, including maliciously chosen field elements as well as malformed messages. The Verifier should validate that (a) field elements are encoded as expected and (b) the message flow follows the description below.)

See <a href="#spartan-full-image">figure below</a> for a pictorial description: 

#### Sumcheck-1

> 1. $\mathcal{P} \longrightarrow \mathcal{V}$: Prover computes commitments to $(\mle{A}, \mle{B}, \mle{C})$ and $\mle{w}$ and sends them to the Verifier.
> 2. $\mathcal{V} \longrightarrow \mathcal{P}$: The Verifier sends a uniformly random challenge $\vec{r}_\tau \in \mathbb{F}^s$ to the Prover, and expects the Prover to prove the following claim using Sumcheck: $$0 = \sum_{\vec{x}\in \{0,1\}^s}\mathcal{G}(\vec{x}, \vec{r}_\tau)$$
> 3. $\mathcal{P} \longleftrightarrow \mathcal{V}$: Prover and the Verifier exchange Sumcheck messages as usual, during which:
>       * The Verifier uniformly samples $\vec{r}_\Sigma \in \mathbb{F}^s$ random field elements for total $s$ rounds, and 
>       * The Verifier checks that the maximum degree of univariate polynomials sent by the Prover never exceed three.
>
>       * At the end of the Sumcheck round, the Prover's claim about the sum over the boolean hypercube will reduce to a claim that $\mathcal{G}(\vec{r}_\Sigma, \vec{r}_\tau)$ evaluates to some field element $G$, or in Sumcheck notation: $$ 0 \stackrel{?}{=} \sum_{\vec{x} \in \{0,1\}^s} \mathcal{G}(\vec{x}, \vec{r}_\tau) \stackrel{\vec{r}_\Sigma}{\implies} G \stackrel{?}{=} \mathcal{G}(\vec{r}_\Sigma, \vec{r}_\tau)$$
>         The verifier needs to independently evaluate $\mathcal{G}(\vec{r}_\Sigma, \vec{r}_\tau)$ to validate the Prover's claim about $G$.
> 4. $\mathcal{P} \longrightarrow \mathcal{V}$: To facilitate the computation of $\mathcal{G}(\vec{r}_\Sigma, \vec{r}_\tau)$ an honest Prover computes $$\begin{aligned}v_A &= \overline{A}(\vec{r}_\Sigma) := \sum_{j \in [m]}\mle{A}(\vec{r}_\Sigma, \vec{j})\cdot\mle{w}(\vec{j}) \\ v_B &= \overline{B}(\vec{r}_\Sigma) := \sum_{j \in [m]}\mle{B}(\vec{r}_\Sigma, \vec{j})\cdot\mle{w}(\vec{j}) \\ v_C &= \overline{C}(\vec{r}_\Sigma) := \sum_{j \in [m]}\mle{C}(\vec{r}_\Sigma, \vec{j})\cdot\mle{w}(\vec{j}) \end{aligned}$$
>      and sends $(v_A,v_B,v_C) \in \mathbb{F}^3$ to the Verifier.


Given $(v_A,v_B,v_C)$, the verifier can compute $\mathcal{G}(\vec{r}_\Sigma,\vec{r}_\tau)$ locally using the following relations: $$\begin{aligned}\mathcal{G}(\vec{x},\vec{\tau}) &:= \hat{f}(\vec{x})\cdot\eq(\vec{x},\vec{\tau}) \\ \hat{f}(\vec{x}) &:= \overline{A}(\vec{x})\cdot\overline{B}(\vec{x}) - \overline{C}(\vec{x}) \end{aligned}$$
    Concretely, the Verifier locally computes (efficiently) $$E = \eq(\vec{r}_\Sigma,\vec{r}_\tau)\stackrel{\text{w.h.p}}{\neq} 0$$ and validates that Prover's final round Sumcheck claim $G$ satisfies $$ G \stackrel{?}{=} (v_A\cdot v_B - v_C)\cdot E$$
 
After performing the above validation, the Prover's claim about $(v_A, v_B, v_C)$ are still unverified. To address this, Spartan uses batched Sumcheck to validate these claims. The batched polynomial is defined as: $$M_{\vec{r}_\Sigma}(\vec{y}, \alpha, \beta, \gamma) := \mle{w}(\vec{y})\cdot\left[\alpha\cdot \mle{A}(\vec{r}_\Sigma, \vec{y}) + \beta\cdot \mle{B}(\vec{r}_\Sigma, \vec{y}) + \gamma\cdot \mle{A}(\vec{r}_\Sigma, \vec{y})\right]\in \Fx{2}[\alpha, \beta, \gamma;\;y_1,\cdots,y_s]$$

The Verifier expects the Prover to prove the following claim: $$\forall \alpha,\beta,\gamma \in \mathbb{F}:\quad V(\alpha,\beta, \gamma) := \alpha\cdot v_A + \beta\cdot v_B + \gamma\cdot v_C \stackrel{?}{=} \sum_{\vec{y} \in \{0,1\}^s}M_{\vec{r}_\Sigma}(\vec{y}, \alpha, \beta, \gamma)$$ Notice that $V(\alpha,\beta,\gamma)$ is linear, and the above equation can only be satisfied for _all values_ of $\alpha, \beta$, and $\gamma$, if $v_A, v_B$, and $v_C$ satisfy the equation in step-4. The Verifier invokes the Schwartz–Zippel lemma on $V(\alpha,\beta,\gamma)$ to validate the above claim at the cost of soundness loss of $O(\frac{1}{|\mathbb{F}|})$.

The following protocol describes the details of rest of the protocol:

#### Sumcheck-2

> 5. $\mathcal{V} \longrightarrow \mathcal{P}$: The Verifier uniformly samples three random field elements $r_A, r_B, r_C \xleftarrow{\$} \mathbb{F}$ and sends them to the Prover. The Verifier locally computes $$V := V(r_A,r_B, r_C) = r_A\cdot v_A + r_B\cdot v_B + r_C\cdot v_C$$ and expects the Prover to prove the following claim using Sumcheck: $$ V \stackrel{?}{=} \sum_{\vec{y} \in \{0,1\}^s} M_{\vec{r}_\Sigma}(\vec{y}, r_A,r_B, r_C)$$ and expects the maximum individual degree of $M_{\vec{r}_\Sigma}(\vec{y}, r_A,r_B, r_C)$ to be $2$ during Sumcheck rounds. (Recall: $\mle{w}(\vec{y})$ and $\mle{A}(\vec{r}_\Sigma, \vec{y})$ are multilinear in $\vec{y}$).
> 6. $\mathcal{P} \longleftrightarrow \mathcal{V}$: Prover and the Verifier exchange Sumcheck messages as usual, during which:
>       * The Verifier uniformly samples $\vec{r}_y \in \mathbb{F}^s$ random field elements for total $s$ rounds, and 
>       * The Verifier checks that the maximum degree of univariate polynomials sent by the Prover never exceeds $2$.
>       * At the end of the Sumcheck round, the Prover's claim about the sum over boolean hypercube will reduce to a claim that $M_{\vec{r}_\Sigma}(\vec{r}_y, r_A, r_B, r_C)$ evaluates to some field element $M$, i.e.,: $$ V \stackrel{?}{=} \sum_{\vec{y} \in \{0,1\}^s} M_{\vec{r}_\Sigma}(\vec{y}, r_A,r_B, r_C) \stackrel{\vec{r}_y}{\implies} M \stackrel{?}{=} M_{\vec{r}_\Sigma}(\vec{r}_y, r_A, r_B, r_C)$$

The Verifier needs to independently evaluate $M_{\vec{r}_\Sigma}(\vec{r}_y, r_A, r_B, r_C)$ to validate the Prover's claim. To do that, it uses the polynomial commitments from step-1 to compute $M_{\vec{r}_\Sigma}(\vec{r}_y, r_A, r_B, r_C)$. In more detail:

#### PCS Opening and Final Validation

> 7. $\mathcal{V} \longrightarrow \mathcal{P}$: The Verifier instructs the Prover to open the commitment for $(\mle{w}, \mle{A}, \mle{B}, \mle{C})$ at points $(\vec{x} := \vec{r}_\Sigma, \vec{y} := \vec{r}_y)$.
> 8. $\mathcal{P} \longrightarrow \mathcal{V}$: The Prover computes the following  $$\begin{aligned}
\hat{v}_w &= \mle{w}(\vec{r}_y) \\
\hat{v}_A &= \mle{A}(\vec{r}_\Sigma, \vec{r}_y) \\
\hat{v}_B &= \mle{B}(\vec{r}_\Sigma, \vec{r}_y) \\
\hat{v}_C &= \mle{C}(\vec{r}_\Sigma, \vec{r}_y) \\
\end{aligned}$$ and sends $(\hat{v}_w,\hat{v}_A,\hat{v}_B,\hat{v}_C) \in \mathbb{F}^4$ to the Verifier.
> 9. $\mathcal{P} \longleftrightarrow \mathcal{V}$: The Prover and the Verifier use the PCS opening protocol to validate that $(\hat{v}_w,\hat{v}_A,\hat{v}_B,\hat{v}_C)$ is indeed a valid opening.
> 10. $\mathcal{V}$: The Verifier makes the following to $\textsf{accept}$ or $\textsf{reject}$ the proof: $$M \stackrel{?}{=} \hat{v}_w\cdot \left[ r_A\cdot \hat{v}_A + r_B\cdot \hat{v}_B + r_C\cdot \hat{v}_C\right]$$
> 

The following figure summarizes the protocol pictorially:

<img id="spartan-full-image" src="./spartan/Spartan.drawio.svg" style="margin-left:auto; margin-right:auto"/>

#### Soundness of Full Spartan

Following steps of the protocol contribute to the soundness loss:

* **Step-2**: Uses Schwartz-Zippel to conclude that $\forall \vec{\tau}\in\mathbb{F}^s:\;\; \sum_{\vec{x}\in \{0,1\}^s} \mathcal{G}(\vec{x},\vec{\tau}) = 0$. This incurs a soundness loss of $O(\frac{1}{|\mathbb{F}|})$. To enforce soundness loss that's less than what's acceptable for security parameter $\lambda$, i.e., for $\frac{1}{|\mathbb{F}|} < 2^{-\lambda}$, the field size must be appropriately large, i.e., $|\mathbb{F}| > \Omega({2^\lambda})$. 
* **Step-3**: Uses Sumcheck over $s$-variate polynomials of maximum individual degree $3$. This incurs a soundness loss of $\frac{3}{\mathbb{F}}\cdot s$. Security parameter requirement: $|\mathbb{F}| > \Omega(s\cdot {2^\lambda})$.
* **Step-5**: Uses Schwartz-Zippel to conclude that $\forall \alpha,\beta,\gamma \in \mathbb{F}:\;\;\alpha\cdot v_A + \beta\cdot v_B + \gamma\cdot v_C = \sum_{\vec{y} \in \{0,1\}^s}M_{\vec{r}_\Sigma}(\vec{y}, \alpha, \beta, \gamma)$. This incurs a soundness loss of $O(\frac{1}{|\mathbb{F}|}).$ Security parameter requirement: $|\mathbb{F}| > \Omega({2^\lambda})$
* **Step-6**: Uses Sumcheck over $s$-variate polynomials of maximum individual degree $2$. This incurs a soundness loss of $\frac{2}{\mathbb{F}}\cdot s \in O(\frac{s}{|\mathbb{F}|})$. Security parameter requirement: $|\mathbb{F}| > \Omega(s\cdot {2^\lambda})$.
* **Steps-{7:10}**: Finally, assuming that the PCS has a soundness loss of $\delta_\lambda(\nu)$ for a $\nu$-variate polynomial, by union bound, the four opening proofs of PCS incur a soundness loss of $$3\cdot \delta_\lambda(2\cdot s) + \delta_\lambda(s)$$

This leads to a total soundness loss of $$ O\left(\frac{s}{|\mathbb{F}|} \right ) + 3\cdot \delta_\lambda(2\cdot s) + \delta_\lambda(s)$$ and a constraint on the field size of $$|\mathbb{F}| > \Omega\left(\max\left\{s\cdot 2^\lambda,\;\frac{1}{3\cdot \delta_\lambda(2\cdot s) + \delta_\lambda(s)}\right\}\right)$$

Note that the polynomial commitment for $\mle{A}, \mle{B}, \mle{C}$ are defined over $2s$-variate multilinear polynomials. Even though the two Sumchecks always run over an $s$-variate polynomial, the scheme as described above, still requires $2s$-variate PCS.

## Sample Code

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('./sage-snark'))
sys.path.insert(0, module_path)

### Arithmetization in Sagemath

The following code shows examples of Spartan like arithmetization over the prime field $\mathbb{F} := 15\cdot 2^{27} + 1$. It first generates a random R1CS instance from the [`utils.test_utils`](./utils/test_utils.py) package and then uses `matrix_multilinearize` method from [`utils.multivairates`](./utils/multivariates.py) to generate a multilinear polynomial whose coefficients are same as the Matrix elements.

In [ ]:
from utils.test_utils import R1CS
from sage.rings.finite_rings.all import GF

In [ ]:
P = 15*(2**27) + 1  # Proth prime with nice multiplicative subgroups
Fq = GF(P)

# Create a random R1CS instance that's satisfiable
r1cs_instance = R1CS.random_element(Fq, num_rows=5, num_columns=7)

# Check the instance is satisfiable
assert r1cs_instance.is_satisfiable()

A = r1cs_instance.A
B = r1cs_instance.B
C = r1cs_instance.C
w = r1cs_instance.w

# Generate the minimal Spartan Multilinear polynomial.
# The result is polynomial, and it's parent polynomial Ring
# can be re-used with other Matrices
Axy = r1cs_instance.a_tilde()
Bxy = r1cs_instance.b_tilde()
Cxy = r1cs_instance.c_tilde()
wy = r1cs_instance.w_tilde()

In [ ]:
print(f"{A}")
print(f"\nMultilinear extension: {Axy}")

In [ ]:
print(f"{B}")
print(f"\nMultilinear extension: {Bxy}")

In [ ]:
print(f"{C}")
print(f"\nMultilinear extension: {Cxy}")

In [ ]:
print(f"{w}")
print(f"{wy}")

In [ ]:
# Sample R1CS Instance. Taken from: https://emirsoyturk.medium.com/hello-arithmetization-55e57c8e5471

AL = [
     [0, 1, 0, 0, 0, 0],
     [0, 0, 0, 1, 0, 0],
     [0, 1, 0, 0, 1, 0],
     [5, 0, 0, 0, 0, 1]
    ]

BL = [
     [0, 1, 0, 0, 0, 0],
     [0, 1, 0, 0, 0, 0],
     [1, 0, 0, 0, 0, 0],
     [1, 0, 0, 0, 0, 0]
    ]

CL = [
     [0, 0, 0, 1, 0, 0],
     [0, 0, 0, 0, 1, 0],
     [0, 0, 0, 0, 0, 1],
     [0, 0, 1, 0, 0, 0]
    ]

WL = [1, 3, 35, 9, 27, 30]

In [ ]:
def compute_y_sum(poly, y_dim):
    gens = poly.parent().gens()
    skip_len = len(gens) - Integer(y_dim).bit_length()
    assert skip_len >= 0

    skip_vars = gens[:skip_len]
    return hypercube_sum(poly, skip_vars)

In [ ]:
P = 15*(2**27) + 1;
assert is_prime(P)
Fp = GF(P)

A = Matrix(Fp, AL)
B = Matrix(Fp, BL)
C = Matrix(Fp, CL)
w = vector(Fp, WL);
aw = A*w;
bw = B*w;
cw = C*w;

# print(f"W = {w}")
# print(f"A*W = {aw}")
# print(f"B*W = {bw}")
# print(f"C*W = {cw}")

assert list(cw) == hadamard_product(aw, bw)

In [ ]:
Axy = matrix_multilinearize(A);
Bxy = matrix_multilinearize(B, Axy.parent())
Cxy = matrix_multilinearize(C, Axy.parent())

xgens_count = Integer(A.nrows()).bit_length()
xgens = Axy.parent().gens()[:xgens_count];
ygens = Axy.parent().gens()[xgens_count:];

Wy = vec_multilinearize(w, ygens)

Ax = compute_y_sum(Axy*Wy, 6)
Bx = compute_y_sum(Bxy*Wy, 6)
Cx = compute_y_sum(Cxy*Wy, 6)

expected = Ax*Bx - Cx

for i in range(A.nrows()):
    d = bit_decomp_dict(i, xgens)
    e = expected.subs(d)
    assert e == 0
